# Analyze Fine-Tuned EfficientPhys Models

## Load the Predictions

In [ ]:
import numpy as np
import pandas as pd
import respiration.utils as utils

signals_path = utils.file_path('outputs', 'signals')
predictions_path = utils.join_paths(signals_path, 'fine_tuned_predictions.csv')

prediction = pd.read_csv(predictions_path)
prediction['signal'] = prediction['signal'].apply(eval).apply(np.array)

In [ ]:
prediction.head()

## Get the Ground Truth RR Signal

In [ ]:
ground_truth_file = utils.join_paths(signals_path, 'respiration_gt.csv')
ground_truth = pd.read_csv(ground_truth_file)
ground_truth['signal'] = ground_truth['signal'].apply(eval).apply(np.array)
ground_truth.head()

## Analyze the Performance of the Model

In [ ]:
models = prediction['model'].unique()
models

In [ ]:
# The testing scenarios are the same for each model
manifest_path = utils.file_path('models', 'fine_tuned', '20240509_122327', 'manifest.json')
manifest = utils.read_json(manifest_path)
testing_scenarios = manifest['testing_scenarios']
testing_scenarios

In [ ]:
import os
from tqdm.auto import tqdm
import respiration.analysis as analysis

analysis_dir = utils.dir_path('outputs', 'analysis', mkdir=True)
analysis_file = os.path.join(analysis_dir, 'fine_tuned_analysis.csv')

if os.path.exists(analysis_file):
    analysis_results = pd.read_csv(analysis_file)
else:
    analysis_results = []

    for model_id in tqdm(models):
        for (subject, setting) in testing_scenarios:
            scenario = prediction[(prediction['model'] == model_id) &
                                  (prediction['subject'] == subject) &
                                  (prediction['setting'] == setting)].iloc[0]
            prediction_signal = scenario['signal']

            scenario_gt = ground_truth[(ground_truth['subject'] == subject) &
                                       (ground_truth['setting'] == setting)].iloc[0]

            # Cut the signal to the same length
            gt_signal = scenario_gt['signal'][1:len(prediction_signal) + 1]

            comparator = analysis.SignalComparator(
                prediction_signal,
                gt_signal,
                scenario['sampling_rate'],
            )

            for metric, result in comparator.all_results().items():
                analysis_results.append({
                    'model_id': model_id,
                    'subject': subject,
                    'setting': setting,
                    'metric': metric,
                    'prediction': result['prediction'],
                    'ground_truth': result['ground_truth'],
                })

    analysis_results = pd.DataFrame(analysis_results)
    analysis_results.to_csv(analysis_file, index=False)

In [ ]:
analysis_results.head()

In [ ]:
analysis_results['error'] = (analysis_results['prediction'] - analysis_results['ground_truth']).abs()

In [ ]:
# Plot the pk metrics error for each model
import matplotlib.pyplot as plt

models = prediction['model'].unique()
metrics = analysis_results['metric'].unique()

fig = plt.figure(figsize=(20, 5))
for model_id in models:
    model_results = analysis_results[(analysis_results['model_id'] == model_id)]

    # Plot the mean error for each model
    mean_error = model_results['error'].mean()
    plt.bar(model_id, mean_error, label=model_id)

plt.xlabel('Model')
plt.ylabel('Mean Error')
plt.title('Mean Error for Fine-Tuned Models')

plt.show()